In [1]:
import numpy as np
import psycopg2
import datetime
import math

In [2]:
npyname = "kddi_12_param_new_result"
sttime = "2022-04-01 12:00:00"

In [12]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [4]:
#道路情報に重なるメッシュIDをすべて取得
sql = "SELECT osm_id FROM roadmap"
cur =connection.cursor()
cur.execute(sql)
keylist = []
for row in cur:
    sql = "SELECT a.grid_code"
    sql += " FROM pointdata a , roadmap r "
    sql += " WHERE ST_Distance(a.geom, r.geom) < 0.003 "
    sql += " AND r.osm_id = " + str(row[0])

    cur2 =connection.cursor()
    cur2.execute(sql)
    childlist = []
    for row2 in cur2:
        childlist.append(row2[0])
    cur2.close()
    keylist.append([str(row[0]), childlist])
cur.close()

In [5]:
#道路ID, メッシュID
keylist

[['27755814',
  ['50324595243',
   '50324595244',
   '50324595241',
   '50324595242',
   '50324596131',
   '50324596132',
   '50324595214',
   '50324595223',
   '50324595224',
   '50324596113',
   '50324596114',
   '50324595212',
   '50324595221',
   '50324595222',
   '50324596111',
   '50324596112',
   '50324596121',
   '50324585443',
   '50324585444',
   '50324586333',
   '50324586334',
   '50324586343',
   '50324585441',
   '50324585442',
   '50324586331',
   '50324586332',
   '50324586341',
   '50324585424',
   '50324586313',
   '50324586314',
   '50324586323',
   '50324585422',
   '50324586311',
   '50324586312',
   '50324586133',
   '50324586134']],
 ['27755914',
  ['50324525414',
   '50324525423',
   '50324525411',
   '50324525412',
   '50324525421',
   '50324525422',
   '50324525233',
   '50324525234',
   '50324525243',
   '50324525244']],
 ['27755921', []],
 ['27755953', []],
 ['27808255',
  ['50326507413',
   '50326507414',
   '50326507423',
   '50326507424',
   '50326507411'

In [6]:
len(keylist)

3715

In [7]:
#データが存在する道路IDごとに交通量を平均化して取得する
cntDt = 0
wks = ""
wknp=""
find = 0
dtstart = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')

cnts = 0
for row in keylist:
    if cnts > 3500:
        cnts = cnts + 1
        continue
    if cnts > 3750:
        break
    cnts = cnts + 1
    if wks == "":
        wks = row[0]
    elif wks != row[0]:
        if cntDt != 0:
            if find != 0:
                a_1 = np.array([[wks, np.array(lstItems)]])
                wknp = np.append(wknp, a_1, axis=0)
            else:
                find = 1
                a_1 = np.array([[wks, np.array(lstItems)]])
                wknp = a_1
        wks = row[0]
        cntDt = 0

    lstItems = [-1, -1, -1, -1,  -1,-1, -1, -1,  -1, -1,-1, -1,   -1,-1, -1,-1,   -1,-1, -1, -1 ,  -1, -1, -1, -1]

    print(row[0])
    for meshid in row[1]:
        
        sql = "SELECT times,tbran from " + npyname + " WHERE mesh = " + meshid + " and times < '2022-04-02 00:00:00'  order by times"
        cur =connection.cursor()
        cur.execute(sql)
        finds = 0
        
        for row2 in cur:
            dtnow = datetime.datetime.strptime(str(row2[0]), '%Y-%m-%d %H:%M:%S')
            dtdiff = dtnow - dtstart
            finds = math.floor(dtdiff.seconds / 60 / 15)
            if lstItems[finds] == -1:
                lstItems[finds] = row2[1]
            else:
                lstItems[finds] = (lstItems[finds] + row2[1]) / 2
            cntDt = cntDt + 1
        cur.close()
if cntDt != 0:
    if find != 0:
        a_1 = np.array([[wks, np.array(lstItems)]])
        wknp = np.append(wknp, a_1, axis=0)
    else:
        a_1 = np.array([[wks, np.array(lstItems)]])
        wknp = a_1
print("complete.")


27755814


C:\Users\BIGDATA_DEV\AppData\Local\Temp\ipykernel_7816\3185280707.py:25: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a_1 = np.array([[wks, np.array(lstItems)]])


27755914


C:\Users\BIGDATA_DEV\AppData\Local\Temp\ipykernel_7816\3185280707.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a_1 = np.array([[wks, np.array(lstItems)]])


27755921
27755953
27808255
27808268
38135611
38135612
38135616
38135912
38135927
38135933
38136926
38141295
38141959
46028713
46342987
46342989
46342990
46342991
46344678
46345883
46431629
46431985
46431987
46436571
46436574
46504976
46505493
46505494
46506069
46599220
46599222
46648095
47818676
47825376
47907711
48215486
48215487
48215488
48215491
48215492
48215493
48216247
48217075
48218113
48344601
48381177
48381178
49765900
50123117
50123138
50126684
50126685
51396950
52171216
52171217
52171221
52171222
52171223
52171224
52171225
52171226
52171227
52171249
52171250
52171251
52171252
52171255
52171258
52171259
52458736
52458737
52458743
52458744
52458745
52458746
52458751
52458752
52458753
52460148
52460149
52460150
52460151
52460153
52460154
52463126
52463129
52463130
52463131
52463134
52463135
52463137
52463138
52463139
52463145
52463146
52463149
52463153
54233862
54818301
54818302
54818304
54818306
54818308
54818310
54818313
54818317
54818318
54818320
54818321
54818322
54851978
5

155144591
155144778
155144783
155144855
155229940
155229948
155229950
155229951
155229953
155229955
155229956
155229957
155229958
155229959
155229960
155229961
155229962
155229963
155231788
155231814
155247185
155247191
155247349
155247353
155247398
155372030
155372043
155372082
155372092
155372115
155372116
155372120
155372138
155372157
155372158
155372162
155372166
155372226
155372240
155372247
155372261
155372270
155372276
155372277
155372324
155372327
155372350
155372354
155372409
155372451
155372509
155373418
155373425
155373440
155377136
155382677
155382728
155382748
155382795
155382858
155382932
155382964
155383065
155383111
155383113
155383144
155383157
155383176
155383178
155383179
155383238
155383247
155383268
155383269
155383307
155383327
155383353
155383359
155383361
155383367
155383381
155383403
155383423
155383438
155383443
155383466
155383473
155383523
155383534
155383556
155383559
155383568
155383590
155383594
155383621
155383673
155383699
155383800
155383814
155383905


155709935
155709938
155709965
155709982
155710032
155710880
155710953
155710992
155711013
155711016
155711040
155711069
155711087
155711088
155711091
155711095
155711289
155711307
155711308
155711789
155712110
155712226
155712262
155712299
155712358
155712421
155712451
155713116
155713274
155713277
155713377
155713382
155713484
155713489
155713527
155713613
155713639
155713662
155714246
155714336
155714430
155714529
155714645
155714804
155715076
155715083
155715095
155715247
155715347
155715712
155715785
155715786
155715791
155715795
155715798
155715804
155715811
155715815
155715818
155715822
155715829
155715837
155715846
155715856
155715861
155715865
155715874
155715882
155715890
155715897
155715910
155715912
155715924
155715929
155715936
155715942
155715944
155715948
155939244
155939272
156299322
156557748
156557753
156991668
156991670
156991671
156991674
156991676
156991688
156991691
156991692
158516709
158516710
158516712
158516713
158516714
158516716
158516717
158516720
158516721


294823480
294823481
294823482
294823483
294823484
294823485
294823486
294823487
294823488
294823489
294823490
294897259
294897260
294907697
294907698
294907703
294907704
294907705
294907706
294958825
294958827
294958892
294958893
295047640
295047641
295054821
295054822
295078417
295078419
295825707
296018242
296018244
296122704
296122705
296188860
296189009
296367168
296367169
296386806
296386807
296391473
296391474
296391475
296391476
296392097
296392098
296399390
296399391
296399392
296399393
296399394
296399395
296579317
296601775
296601776
296604594
296667146
296667147
296784461
296784462
297130200
297735866
298932605
298932606
300240691
300240693
302013365
304570718
304570719
306736529
306736536
306736538
306736541
306736550
306736555
306736566
306736568
306736570
306736571
306736572
306736573
306736574
306736575
306736576
306736577
306736578
306736579
306736580
306736581
306736583
306736585
306736587
306736588
306736589
306736590
306736591
306736592
306736593
306736594
306736597


636996189
636997542
636998607
637001114
637001115
637001333
637048728
637108637
637117097
637123033
637123034
637123036
637123038
637124106
637124107
637140509
637145956
637153192
637167088
637169016
637190051
637191741
637208100
637229817
637231513
637231990
637237689
637241706
637456984
639214481
642814900
642814901
642814902
642814903
648461983
648461986
648461999
648462000
648462001
648462002
648487854
651358174
651358177
651358180
651358203
652928507
652928508
652928509
652928510
652928512
652928513
652928514
652928515
652928517
652928518
654133590
654133591
654133592
654133594
654853420
656284808
659278117
659278118
659449604
659450373
659688060
659835297
659835298
659835299
659835300
659835301
659835302
660389633
660389635
660446122
660446123
660458840
660458841
668319026
668319030
668319034
668319035
668319037
668319042
668319047
668319048
668319051
668319052
668319053
668319054
668319057
668319058
668319059
668319061
668319062
668319063
668319064
668319065
668319066
668319068


In [8]:
connection.close()

In [9]:
wknp

array([['27755814',
        array([0.68723914, 0.98829777, 0.36974581, 0.5237848 , 0.58404634,
               0.96351709, 1.26395729, 1.48722406, 0.79648185, 2.17760511,
               0.38871161, 2.37657285, 0.27672237, 2.31304747, 3.40398193,
               0.63575288, 3.75024392, 0.79267801, 0.55175535, 5.76762088,
               1.7223891 , 6.51912958, 2.47411954, 6.86409805])           ],
       ['27755914',
        array([0.92542148, 1.16430501, 0.70436894, 0.66731327, 0.71364717,
               0.47027655, 0.83027764, 0.46029123, 0.64225249, 2.85697371,
               0.9927812 , 1.62416291, 0.8680952 , 5.58291283, 5.8836491 ,
               1.18653636, 2.00346674, 1.0904982 , 1.59006305, 3.86318078,
               1.38237426, 4.5663628 , 1.97974381, 3.51709585])           ],
       ['27808255',
        array([ 3.06869483,  3.2969602 ,  2.42061433,  2.78619105,  2.90366222,
                2.8991329 ,  2.80293355,  5.16046357,  1.72567067, 11.16843941,
                2.3555471 

In [10]:
np.savetxt(npyname + '_loaddata6.csv',wknp,fmt="%s", delimiter='\t')

In [13]:
sql = "CREATE TABLE " + npyname + "_loadsummary6 (times TIMESTAMP NULL DEFAULT NULL,osm_id BIGINT NULL DEFAULT NULL,tbran DOUBLE PRECISION NULL DEFAULT NULL )"
cur =connection.cursor()
cur.execute(sql)
cur.close()
connection.commit()


In [14]:
cur =connection.cursor()

for items in wknp:
    dte = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')
    for params in items[1]:
        sql = "INSERT INTO " + npyname + "_loadsummary6 (times, osm_id,tbran) VALUES ('"+ str(dte) + "',"  + str(items[0]) + "," + str(params) + ");"
        cur.execute(sql)
        dte = dte + datetime.timedelta(minutes=15)
cur.close()
connection.commit()

In [ ]:
#CreateGeojsonFile